# Missing Meta Link Analysis

In [1]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Analysis by Object

In [2]:
objects_w_missing_metadata = {o: o.missing_meta_links for o in Object.all_objects() if o.missing_meta_links}

In [3]:
len(objects_w_missing_metadata)

457

In [4]:
import collections
by_type = collections.Counter(mt for missing_types in objects_w_missing_metadata.values() for mt in missing_types)
print(by_type.most_common())

[('subpart', 290), ('fillable', 167)]


### Split by provider for work

In [5]:
todo = {"togglebutton", "slicer", "heatsource", "fluidsource", "fluidsink"}
by_provider = collections.defaultdict(dict)
for o, mts in objects_w_missing_metadata.items():
    required_mts = todo.intersection(mts)
    if not required_mts:
        continue

    by_provider[o.provider][o.name] = required_mts

for provider, objects in sorted(by_provider.items()):
    print(provider)
    for name, mts in objects.items():
        print(f"  {name}: {mts}")
    print()

## Analysis by Synset

In [6]:
synsets_w_missing_metadata = [s for s in Synset.all_objects() if not s.has_fully_supporting_object]
print(len(synsets_w_missing_metadata))

402


In [7]:
tr_synsets_w_missing_metadata = [s for s in Synset.all_objects() if len(s.tasks) > 0 and not s.has_fully_supporting_object]
print(len(tr_synsets_w_missing_metadata))
print(tr_synsets_w_missing_metadata)

34
[Synset(name='ashtray.n.01'), Synset(name='bag.n.04'), Synset(name='barbecue_sauce__bottle.n.01'), Synset(name='barrow.n.03'), Synset(name='bath_towel.n.01'), Synset(name='beer_barrel.n.01'), Synset(name='bird_feeder.n.01'), Synset(name='catsup__bottle.n.01'), Synset(name='chocolate_sauce__bottle.n.01'), Synset(name='compost_bin.n.01'), Synset(name='cookie_sheet.n.01'), Synset(name='copper_pot.n.01'), Synset(name='glaze__bottle.n.01'), Synset(name='hot_sauce__bottle.n.01'), Synset(name='litter_box.n.01'), Synset(name='mixing_bowl.n.01'), Synset(name='mustard__bottle.n.01'), Synset(name='napkin.n.01'), Synset(name='person.n.01'), Synset(name='punching_bag.n.02'), Synset(name='sack.n.01'), Synset(name='salsa__bottle.n.01'), Synset(name='saucepan.n.01'), Synset(name='soy_sauce__bottle.n.01'), Synset(name='sugar_syrup__bottle.n.01'), Synset(name='sweater.n.01'), Synset(name='table_linen.n.01'), Synset(name='tissue.n.02'), Synset(name='towel.n.01'), Synset(name='underwear.n.01'), Synset(

In [8]:
# Check if any sliceable task relevant synsets are bad
def get_sliced(s):
  return Synset.get("half__" + s.name.split(".n.")[0] + ".n.01")

transition_relevant_synsets = {
  s
  for t in Task.all_objects()
  for transition in t.relevant_transitions
  for s in list(transition.output_synsets) + list(transition.input_synsets)
}

sliceable_tr_synsets_w_missing_metadata = [
  (s, get_sliced(s)) for s in Synset.all_objects()
  if (
    any(p.name == "sliceable" for p in s.properties) and
    not any(ml.name == "subpart" for o in s.matching_objects for ml in o.meta_links)) and
    (len(get_sliced(s).tasks) > 0 or s in transition_relevant_synsets)]
print(len(sliceable_tr_synsets_w_missing_metadata), "missing subpart annotation despite having sliceable part")
print("\n".join(f"{s.name} -> {sliced.name}" for s, sliced in sliceable_tr_synsets_w_missing_metadata))
print()
missing_half_objs = [sliced for s, sliced in sliceable_tr_synsets_w_missing_metadata if len(sliced.matching_objects) == 0]
print(len(missing_half_objs), "missing half objs")
print(missing_half_objs)

0 missing subpart annotation despite having sliceable part


0 missing half objs
[]


In [9]:
clam = Synset.get("clam.n.03")
for task in Task.all_objects():
  for transition in task.relevant_transitions:
    if clam in transition.input_synsets or clam in transition.output_synsets:
      print(task.name, transition.name, transition.input_synsets, transition.output_synsets)

make_seafood_stew-0 fish_stew <bddl.knowledge_base.orm.ManyToMany object at 0x7f653886cc40> <bddl.knowledge_base.orm.ManyToMany object at 0x7f653886ccd0>


In [10]:
# Check if any sliceable task relevant synsets are bad
sliceable_synsets_w_missing_metadata = [
  (s, get_sliced(s)) for s in Synset.all_objects()
  if (
    any(p.name == "sliceable" for p in s.properties) and
    len(s.matching_objects) > 0 and
    not any(ml.name == "subpart" for o in s.matching_objects for ml in o.meta_links))
]
print(len(sliceable_synsets_w_missing_metadata), "missing subpart annotation despite having sliceable part")
print("\n".join(f"{s.name} -> {sliced.name}" for s, sliced in sliceable_synsets_w_missing_metadata))
print()
missing_half_objs = [sliced for s, sliced in sliceable_synsets_w_missing_metadata if len(sliced.matching_objects) == 0]
print(len(missing_half_objs), "missing half objs")
print(missing_half_objs)

0 missing subpart annotation despite having sliceable part


0 missing half objs
[]


In [11]:
# What objects are fillable?
fillable_objs = [
  o for o in Object.all_objects()
  if any(p.name == 'fillable' for p in o.category.synset.properties)
]
print(len(fillable_objs))
print(sorted([x.name for x in fillable_objs]))

622
['ashtray-dhkkfo', 'ashtray-nfuxzd', 'baking_powder_jar-lgopij', 'baking_sheet-yhurut', 'barbecue_sauce_bottle-gfxrnj', 'basil_jar-swytaw', 'beaker-dtjmai', 'beaker-effbnc', 'beaker-exzsal', 'beaker-fxrsyi', 'beaker-fyrkzs', 'beaker-jdijek', 'beaker-qwoqqr', 'beaker-rhohgs', 'beaker-sfvswx', 'beaker-sstojv', 'beaker-uobdoq', 'beaker-uzgibd', 'beaker-zycgen', 'beef_broth_carton-ecqxgd', 'beer_bottle-nigfha', 'beer_glass-lymciz', 'beer_glass-mxsliu', 'beer_glass-rxscji', 'beer_keg-xtqbuf', 'bird_feed_bag-dpxnlc', 'bird_feeder-hvlkjx', 'blender-cwkvib', 'blender-dhfqid', 'blender-eyedvd', 'blender-xnjqix', 'bowl-adciys', 'bowl-ajzltc', 'bowl-aspeds', 'bowl-belcml', 'bowl-bexgtn', 'bowl-bnobdx', 'bowl-byzaxy', 'bowl-ckxwea', 'bowl-cypjlv', 'bowl-dalyim', 'bowl-eawgwj', 'bowl-eipwho', 'bowl-fedafr', 'bowl-feuaak', 'bowl-fiarri', 'bowl-fwdfeg', 'bowl-hitnkv', 'bowl-hpqjug', 'bowl-hynhgz', 'bowl-jblalf', 'bowl-jfvjep', 'bowl-jhtxxh', 'bowl-jpvcjv', 'bowl-kasebx', 'bowl-kdkrov', 'bowl-kthv

In [ ]:
# Check what synsets don't have full versions
